In [1]:
# importaremos las librerias necesarias
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import Row
from pyspark.sql import functions as F
from pyspark.ml import Pipeline
from pyspark import SparkConf
import os
import shutil
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import col, radians, asin, sin, sqrt, cos, max, min, lit,to_timestamp, split
from pyspark.sql import Window as W
import geopandas as gpd
from shapely import wkt
from pandas_profiling import ProfileReport

In [ ]:
!pyspark --version

In [2]:
spark = SparkSession.builder.master("local[*]").config('spark.driver.extraClassPath', 
 'oracleJDBC/ojdbc8.jar:mssql-jdbc-10.2.1.jre11.jar').config("spark.driver.memory", "3g").getOrCreate()

In [39]:
spark.sparkContext.getConf().getAll()


[('spark.driver.host', '192.168.5.103'),
 ('spark.rdd.compress', 'True'),
 ('spark.driver.extraJavaOptions',
  '-XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED'),
 ('spark.driver.port', '41189'),
 ('spark.driver.extraClassPath',
  'oracleJDBC/ojdbc8.jar:mssql-jdbc-10.2.1.jre11.jar'),
 ('spark.master', 'local[*]'),
 ('spa

In [5]:
# MSSQL connection test

def test_mssql():
csmo_n
    driver = 'com.microsoft.sqlserver.jdbc.SQLServerDriver'
    url = 'jdbc:sqlserver://172.31.0.201:1433'
    server = 'tcp:172.31.0.201' 
    database = 'DWSRBI_KRONOS' 
    username = 'sa' 
    password = 'D@taW@reHous3*' 
    table = '[DWSRBI_KRONOS].[Aux].[Consumos 2013 2021 energuate]'
    # AUX.lineas_consumos_nomralizados
    #VERIFICAR DIMENSION
    consumption_time_series_df = spark.read \
    .format("jdbc") \
    .option("url", url) \
    .option("dbtable", table) \
    .option("user", username) \
    .option("password", password) \
    .option("TrustServerCertificate",True) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .load().limit(100)

    print(consumption_time_series_df.printSchema)

In [6]:


def test_oracle():
    
    driver_oracle = 'oracle.jdbc.driver.OracleDriver'
    url_SGC_ORACLE = 'jdbc:oracle:thin:@10.240.67.143:1521/SGC_TEST'
    username_SGC = 'DR003233' 
    password_oracle_SGC = 'TDV3AC3AA' 
    table = 'SGCPRO.sum_coordenadas' 


    SGC_sum_coordenadas_df = spark.read \
    .format("jdbc") \
    .option("url", url_SGC_ORACLE) \
    .option("dbtable", table) \
    .option("user", username_SGC) \
    .option("driver", driver_oracle) \
    .option("password", password_oracle_SGC) \
    .load().select("NIS_RAD","ALTURA").withColumnRenamed("NIS_RAD","NIS_RAD2").withColumnRenamed("NIS_RAD","NIS_RAD2").limit(50000)

    print(SGC_sum_coordenadas_df.printSchema)

In [7]:
con_series_df = spark.read.parquet("data/fraudes_energuate/consumos_series_2018_2022/part*")
sorted(con_series_df.columns)

['csmo201301',
 'csmo201302',
 'csmo201303',
 'csmo201304',
 'csmo201305',
 'csmo201306',
 'csmo201307',
 'csmo201308',
 'csmo201309',
 'csmo201310',
 'csmo201311',
 'csmo201312',
 'csmo201401',
 'csmo201402',
 'csmo201403',
 'csmo201404',
 'csmo201405',
 'csmo201406',
 'csmo201407',
 'csmo201408',
 'csmo201409',
 'csmo201410',
 'csmo201411',
 'csmo201412',
 'csmo201501',
 'csmo201502',
 'csmo201503',
 'csmo201504',
 'csmo201505',
 'csmo201506',
 'csmo201507',
 'csmo201508',
 'csmo201509',
 'csmo201510',
 'csmo201511',
 'csmo201512',
 'csmo201601',
 'csmo201602',
 'csmo201603',
 'csmo201604',
 'csmo201605',
 'csmo201606',
 'csmo201607',
 'csmo201608',
 'csmo201609',
 'csmo201610',
 'csmo201611',
 'csmo201612',
 'csmo201701',
 'csmo201702',
 'csmo201703',
 'csmo201704',
 'csmo201705',
 'csmo201706',
 'csmo201707',
 'csmo201708',
 'csmo201709',
 'csmo201710',
 'csmo201711',
 'csmo201712',
 'csmo201801',
 'csmo201802',
 'csmo201803',
 'csmo201804',
 'csmo201805',
 'csmo201806',
 'csmo2018

In [8]:
#SGC_sum_coordenadas_df.show()
#print(SGC_sum_coordenadas_df.count(), len(SGC_sum_coordenadas_df.columns))

In [9]:
#SGC_sum_coordenadas_df.show()

In [ ]:
def toParquet():
    csv_Altura_df = spark.read.option("header", True).csv('data/fraudes_energuate/alt.csv')
    NewColumns=(column.replace(' ', '_') for column in csv_fraudes_df.columns)
    csv_Altura_df = csv_Altura_df.toDF(*NewColumns)
    csv_Altura_df.write.parquet("data/fraudes_energuate/parquet_altura_contadores")
    
    csv_fraudes_df = spark.read.option("header", True).csv('data/fraudes_energuate/fraude.csv').withColumnRenamed("ID Servicio","id_servicio")
    NewColumns=(column.replace(' ', '_') for column in csv_fraudes_df.columns)
    csv_fraudes_df = csv_fraudes_df.toDF(*NewColumns)
    csv_fraudes_df.write.parquet("data/fraudes_energuate/parquet_fraudes_2018_2022")

    csv_fraudes_series_df = spark.read.option("header", True).option("delimiter", ";").csv('data/fraudes_energuate/consumos_series.csv')#.withColumnRenamed("ID Servicio","id_servicio")
    NewColumns=(column.replace(' ', '_') for column in csv_fraudes_series_df.columns)
    csv_fraudes_series_df = csv_fraudes_series_df.toDF(*NewColumns)
    csv_fraudes_series_df.write.parquet("data/fraudes_energuate/consumos_series_2018_2022")
    
    csv_map_meters_df = spark.read.option("header", True).csv('data/fraudes_energuate/mapeo_contadores_zonas.csv')
    NewColumns=(column.replace(' ', '_') for column in csv_map_meters_df.columns)
    csv_map_meters_df = csv_map_meters_df.toDF(*NewColumns)
    csv_map_meters_df.write.parquet("data/fraudes_energuate/parquet_mape_contadores")
    

In [23]:
fraudes_df.count()

47466

### Data reading

In [3]:
con_series_df = spark.read.parquet("data/fraudes_energuate/consumos_series_2018_2022/part*").select('nis_rad','csmo201901',
 'csmo201902', 'csmo201903', 'csmo201904', 'csmo201905', 'csmo201906', 'csmo201907', 'csmo201908', 'csmo201909', 'csmo201910', 'csmo201911', 'csmo201912', 'csmo202001', 'csmo202002', 'csmo202003', 'csmo202004', 'csmo202005', 'csmo202006', 'csmo202007', 'csmo202008', 'csmo202009','csmo202010','csmo202011','csmo202012','csmo202101','csmo202102','csmo202103','csmo202104','csmo202105','csmo202106','csmo202107','csmo202108','csmo202109','csmo202110',).dropna()



fraudes_df = spark.read.parquet("data/fraudes_energuate/parquet_fraudes_2018_2022/part*").select('id_servicio','DEPARTAMENTO','MUNICIPIO','LOCALIDAD','Fecha_Deteccion','Fecha_Inicio_Fraude','ID_Centro_Lectura')
split_col = split(fraudes_df['Fecha_Deteccion'], '-')
fraudes_df = fraudes_df.withColumn("fraud_year",split_col.getItem(2)).withColumn("fraude",lit(1)).drop('Fecha_Inicio_Fraude','Fecha_Deteccion')
fraudes_df = fraudes_df.filter((fraudes_df.fraud_year >= 20) & (fraudes_df.fraud_year <= 21)).dropna()


altura_df = spark.read.parquet("data/fraudes_energuate/parquet_altura_contadores/part*").withColumnRenamed('NIS_RAD','nis_rad_2').dropna()


mapeo_contadores = spark.read.parquet("data/fraudes_energuate/parquet_mape_contadores/part*").withColumnRenamed('ID_Servicio','nis_rad_2').dropna()


In [5]:
pd.read_csv("data/fraudes_energuate/consumos_series.csv", sep=';').profile_report(minimal=True).to_file("reporte_no_fraudes_original.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
pd.read_csv("data/fraudes_energuate/consumos_series.csv",sep=';').head()

,nis_rad,csmo201802,csmo201801,csmo201712,csmo201711,csmo201710,csmo201709,csmo201708,csmo201707,csmo201706,...,csmo202101,csmo202102,csmo202103,csmo202104,csmo202105,csmo202106,csmo202107,csmo202108,csmo202109,csmo202110
0,2168144,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2168145,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2168146,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2168147,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2168148,88.0,102.0,92.0,86.0,83.0,88.0,101.0,95.0,94.0,...,118.0,104.0,117.0,117.0,109.0,92.0,115.0,101.0,110.0,82.0


### non fraudulent data

In [4]:
# normal consumption of electricity
df_final_no_fraud = con_series_df.join(fraudes_df, col("nis_rad")==col("id_servicio"), "left_anti")

In [7]:
df_final_no_fraud.printSchema()

root
 |-- nis_rad: string (nullable = true)
 |-- csmo201901: string (nullable = true)
 |-- csmo201902: string (nullable = true)
 |-- csmo201903: string (nullable = true)
 |-- csmo201904: string (nullable = true)
 |-- csmo201905: string (nullable = true)
 |-- csmo201906: string (nullable = true)
 |-- csmo201907: string (nullable = true)
 |-- csmo201908: string (nullable = true)
 |-- csmo201909: string (nullable = true)
 |-- csmo201910: string (nullable = true)
 |-- csmo201911: string (nullable = true)
 |-- csmo201912: string (nullable = true)
 |-- csmo202001: string (nullable = true)
 |-- csmo202002: string (nullable = true)
 |-- csmo202003: string (nullable = true)
 |-- csmo202004: string (nullable = true)
 |-- csmo202005: string (nullable = true)
 |-- csmo202006: string (nullable = true)
 |-- csmo202007: string (nullable = true)
 |-- csmo202008: string (nullable = true)
 |-- csmo202009: string (nullable = true)
 |-- csmo202010: string (nullable = true)
 |-- csmo202011: string (nullabl

In [5]:
join_1 = df_final_no_fraud.join(altura_df,con_series_df.nis_rad ==  altura_df.nis_rad_2,"inner").drop('nis_rad_2')
df_final_no_fraud = join_1.join(mapeo_contadores,con_series_df.nis_rad ==  mapeo_contadores.nis_rad_2,"inner").drop('nis_rad_2').withColumn('fraude',lit(0)).limit(20000)


#df_final_no_fraud = join_2.join(fraudes_df,join_1.nis_rad ==  fraudes_df.id_servicio,"inner")


In [6]:

df_final_no_fraud = df_final_no_fraud.filter(col('csmo201901')!=0) \
.filter(col('csmo201902')!=0).filter(col('csmo201903')!=0).filter(col('csmo201904')!=0).filter(col('csmo201905')!=0) \
.filter(col('csmo201906')!=0).filter(col('csmo201907')!=0).filter(col('csmo201908')!=0).filter(col('csmo201909')!=0) \
.filter(col('csmo201910')!=0).filter(col('csmo201911')!=0).filter(col('csmo201912')!=0).filter(col('csmo202001')!=0) \
.filter(col('csmo202002')!=0).filter(col('csmo202003')!=0).filter(col('csmo202004')!=0).filter(col('csmo202005')!=0) \
.filter(col('csmo202006')!=0).filter(col('csmo202007')!=0).filter(col('csmo202008')!=0).filter(col('csmo202009')!=0) \
.filter(col('csmo202010')!=0).filter(col('csmo202011')!=0).filter(col('csmo202012')!=0).filter(col('csmo202101')!=0) \
.filter(col('csmo202102')!=0).filter(col('csmo202103')!=0).filter(col('csmo202104')!=0).filter(col('csmo202105')!=0) \
.filter(col('csmo202106')!=0).filter(col('csmo202107')!=0).filter(col('csmo202108')!=0).filter(col('csmo202109')!=0) \
.filter(col('csmo202110')!=0) \
.filter(col('csmo201902')!='NaN').filter(col('csmo201903')!='NaN').filter(col('csmo201904')!='NaN').filter(col('csmo201905')!='NaN') \
.filter(col('csmo201906')!='NaN').filter(col('csmo201907')!='NaN').filter(col('csmo201908')!='NaN').filter(col('csmo201909')!='NaN') \
.filter(col('csmo201910')!='NaN').filter(col('csmo201911')!='NaN').filter(col('csmo201912')!='NaN').filter(col('csmo202001')!='NaN') \
.filter(col('csmo202002')!='NaN').filter(col('csmo202003')!='NaN').filter(col('csmo202004')!='NaN').filter(col('csmo202005')!='NaN') \
.filter(col('csmo202006')!='NaN').filter(col('csmo202007')!='NaN').filter(col('csmo202008')!='NaN').filter(col('csmo202009')!='NaN') \
.filter(col('csmo202010')!='NaN').filter(col('csmo202011')!='NaN').filter(col('csmo202012')!='NaN').filter(col('csmo202101')!='NaN') \
.filter(col('csmo202102')!='NaN').filter(col('csmo202103')!='NaN').filter(col('csmo202104')!='NaN').filter(col('csmo202105')!='NaN') \
.filter(col('csmo202106')!='NaN').filter(col('csmo202107')!='NaN').filter(col('csmo202108')!='NaN').filter(col('csmo202109')!='NaN') \
.filter(col('csmo202110')!='NaN') \
.filter(col('csmo201902')!=0.0).filter(col('csmo201903')!=0.0).filter(col('csmo201904')!=0.0).filter(col('csmo201905')!=0.0) \
.filter(col('csmo201906')!=0.0).filter(col('csmo201907')!=0.0).filter(col('csmo201908')!=0.0).filter(col('csmo201909')!=0.0) \
.filter(col('csmo201910')!=0.0).filter(col('csmo201911')!=0.0).filter(col('csmo201912')!=0.0).filter(col('csmo202001')!=0.0) \
.filter(col('csmo202002')!=0.0).filter(col('csmo202003')!=0.0).filter(col('csmo202004')!=0.0).filter(col('csmo202005')!=0.0) \
.filter(col('csmo202006')!=0.0).filter(col('csmo202007')!=0.0).filter(col('csmo202008')!=0.0).filter(col('csmo202009')!=0.0) \
.filter(col('csmo202010')!=0.0).filter(col('csmo202011')!=0.0).filter(col('csmo202012')!=0.0).filter(col('csmo202101')!=0.0) \
.filter(col('csmo202102')!=0.0).filter(col('csmo202103')!=0.0).filter(col('csmo202104')!=0.0).filter(col('csmo202105')!=0.0) \
.filter(col('csmo202106')!=0.0).filter(col('csmo202107')!=0.0).filter(col('csmo202108')!=0.0).filter(col('csmo202109')!=0.0) \
.filter(col('csmo202110')!=0.0)

In [ ]:
df_final_no_fraud.show()

In [9]:
#result_no_fraud.drop("id_servicio").join(fraudes_df, col("nis_rad")==col("id_servicio"), "inner").count()
cols = [F.when(~F.col(x).isin("NULL", "NA", "NaN"), F.col(x)).alias(x)  for x in df_final_no_fraud.columns]
df_final_no_fraud = df_final_no_fraud.select(*cols).dropna()
df_final_no_fraud.toPandas()

,nis_rad,csmo201901,csmo201902,csmo201903,csmo201904,csmo201905,csmo201906,csmo201907,csmo201908,csmo201909,...,csmo202101,csmo202102,csmo202103,csmo202104,csmo202105,csmo202106,csmo202107,csmo202108,csmo202109,csmo202110
0,2168148,125,107,94,90,103,88,81,78,89,...,118,104,117,117,109,92,115,101,110,82
1,2168151,239,167,150,180,136,133,143,137,150,...,165,321,367,464,374,366,363,394,380,391
2,2168155,136,120,147,148,127,127,117,115,129,...,154,163,143,143,138,130,140,147,143,154
3,2168156,3,2,4,6,5,43,21,3,185,...,78,85,79,86,89,86,84,87,78,83
4,2168157,62,55,65,61,60,67,60,58,62,...,61,62,56,65,61,65,63,66,64,70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12202,2188930,167,137,121,131,149,163,158,167,178,...,133,126,126,130,155,16,248,158,169,140
12203,2188933,128,93,105,96,143,118,106,117,114,...,135,140,147,143,185,136,136,139,153,137
12204,2188937,71,75,77,79,83,84,80,85,77,...,101,111,106,109,95,79,90,95,90,98
12205,2188938,41,28,43,38,38,37,43,51,52,...,74,23,44,39,48,43,40,36,57,50


In [7]:
df_final_no_fraud.coalesce(1).write.csv("data/fraudes_energuate/df_final_no_fraud_csv", header = True,mode='overwrite')

In [32]:
df_final_no_fraud.toPandas().profile_report(minimal=True).to_file("reporte_no_fraudes.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [12]:
#fraudes_df.dropna().show()

### Fraudulent data

In [8]:
join_1 = con_series_df.join(altura_df,con_series_df.nis_rad ==  altura_df.nis_rad_2,"inner").drop('nis_rad_2')
df_final = join_1.join(fraudes_df,join_1.nis_rad ==  fraudes_df.id_servicio,"inner")

In [31]:
df_final.toPandas().profile_report(minimal=True).to_file("reporte_fraudes_prepro.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [14]:

cols = [F.when(~F.col(x).isin("NULL", "NA", "NaN"), F.col(x)).alias(x)  for x in df_final.columns]
df_final = df_final.select(*cols).dropna()

In [16]:
df_final_no_fraud.printSchema()

root
 |-- nis_rad: string (nullable = true)
 |-- csmo201901: string (nullable = true)
 |-- csmo201902: string (nullable = true)
 |-- csmo201903: string (nullable = true)
 |-- csmo201904: string (nullable = true)
 |-- csmo201905: string (nullable = true)
 |-- csmo201906: string (nullable = true)
 |-- csmo201907: string (nullable = true)
 |-- csmo201908: string (nullable = true)
 |-- csmo201909: string (nullable = true)
 |-- csmo201910: string (nullable = true)
 |-- csmo201911: string (nullable = true)
 |-- csmo201912: string (nullable = true)
 |-- csmo202001: string (nullable = true)
 |-- csmo202002: string (nullable = true)
 |-- csmo202003: string (nullable = true)
 |-- csmo202004: string (nullable = true)
 |-- csmo202005: string (nullable = true)
 |-- csmo202006: string (nullable = true)
 |-- csmo202007: string (nullable = true)
 |-- csmo202008: string (nullable = true)
 |-- csmo202009: string (nullable = true)
 |-- csmo202010: string (nullable = true)
 |-- csmo202011: string (nullabl

In [9]:
df_final.printSchema()

root
 |-- nis_rad: string (nullable = true)
 |-- csmo201901: string (nullable = true)
 |-- csmo201902: string (nullable = true)
 |-- csmo201903: string (nullable = true)
 |-- csmo201904: string (nullable = true)
 |-- csmo201905: string (nullable = true)
 |-- csmo201906: string (nullable = true)
 |-- csmo201907: string (nullable = true)
 |-- csmo201908: string (nullable = true)
 |-- csmo201909: string (nullable = true)
 |-- csmo201910: string (nullable = true)
 |-- csmo201911: string (nullable = true)
 |-- csmo201912: string (nullable = true)
 |-- csmo202001: string (nullable = true)
 |-- csmo202002: string (nullable = true)
 |-- csmo202003: string (nullable = true)
 |-- csmo202004: string (nullable = true)
 |-- csmo202005: string (nullable = true)
 |-- csmo202006: string (nullable = true)
 |-- csmo202007: string (nullable = true)
 |-- csmo202008: string (nullable = true)
 |-- csmo202009: string (nullable = true)
 |-- csmo202010: string (nullable = true)
 |-- csmo202011: string (nullabl

In [18]:
#join_2 = join_1.join(SGC_sum_coordenadas_df,join_1.nis_rad ==  SGC_sum_coordenadas_df.NIS_RAD2,"inner")

In [19]:
#df_final = join_2.select(sorted(join_2.columns))

In [20]:
#join_1.toPandas()

In [11]:
df_final.printSchema()

root
 |-- nis_rad: string (nullable = true)
 |-- csmo201901: string (nullable = true)
 |-- csmo201902: string (nullable = true)
 |-- csmo201903: string (nullable = true)
 |-- csmo201904: string (nullable = true)
 |-- csmo201905: string (nullable = true)
 |-- csmo201906: string (nullable = true)
 |-- csmo201907: string (nullable = true)
 |-- csmo201908: string (nullable = true)
 |-- csmo201909: string (nullable = true)
 |-- csmo201910: string (nullable = true)
 |-- csmo201911: string (nullable = true)
 |-- csmo201912: string (nullable = true)
 |-- csmo202001: string (nullable = true)
 |-- csmo202002: string (nullable = true)
 |-- csmo202003: string (nullable = true)
 |-- csmo202004: string (nullable = true)
 |-- csmo202005: string (nullable = true)
 |-- csmo202006: string (nullable = true)
 |-- csmo202007: string (nullable = true)
 |-- csmo202008: string (nullable = true)
 |-- csmo202009: string (nullable = true)
 |-- csmo202010: string (nullable = true)
 |-- csmo202011: string (nullabl

In [10]:
df_final = df_final.filter(col('csmo201901')!=0) \
.filter(col('csmo201902')!=0).filter(col('csmo201903')!=0).filter(col('csmo201904')!=0).filter(col('csmo201905')!=0) \
.filter(col('csmo201906')!=0).filter(col('csmo201907')!=0).filter(col('csmo201908')!=0).filter(col('csmo201909')!=0) \
.filter(col('csmo201910')!=0).filter(col('csmo201911')!=0).filter(col('csmo201912')!=0).filter(col('csmo202001')!=0) \
.filter(col('csmo202002')!=0).filter(col('csmo202003')!=0).filter(col('csmo202004')!=0).filter(col('csmo202005')!=0) \
.filter(col('csmo202006')!=0).filter(col('csmo202007')!=0).filter(col('csmo202008')!=0).filter(col('csmo202009')!=0) \
.filter(col('csmo202010')!=0).filter(col('csmo202011')!=0).filter(col('csmo202012')!=0).filter(col('csmo202101')!=0) \
.filter(col('csmo202102')!=0).filter(col('csmo202103')!=0).filter(col('csmo202104')!=0).filter(col('csmo202105')!=0) \
.filter(col('csmo202106')!=0).filter(col('csmo202107')!=0).filter(col('csmo202108')!=0).filter(col('csmo202109')!=0) \
.filter(col('csmo202110')!=0) \
.filter(col('csmo201902')!='NaN').filter(col('csmo201903')!='NaN').filter(col('csmo201904')!='NaN').filter(col('csmo201905')!='NaN') \
.filter(col('csmo201906')!='NaN').filter(col('csmo201907')!='NaN').filter(col('csmo201908')!='NaN').filter(col('csmo201909')!='NaN') \
.filter(col('csmo201910')!='NaN').filter(col('csmo201911')!='NaN').filter(col('csmo201912')!='NaN').filter(col('csmo202001')!='NaN') \
.filter(col('csmo202002')!='NaN').filter(col('csmo202003')!='NaN').filter(col('csmo202004')!='NaN').filter(col('csmo202005')!='NaN') \
.filter(col('csmo202006')!='NaN').filter(col('csmo202007')!='NaN').filter(col('csmo202008')!='NaN').filter(col('csmo202009')!='NaN') \
.filter(col('csmo202010')!='NaN').filter(col('csmo202011')!='NaN').filter(col('csmo202012')!='NaN').filter(col('csmo202101')!='NaN') \
.filter(col('csmo202102')!='NaN').filter(col('csmo202103')!='NaN').filter(col('csmo202104')!='NaN').filter(col('csmo202105')!='NaN') \
.filter(col('csmo202106')!='NaN').filter(col('csmo202107')!='NaN').filter(col('csmo202108')!='NaN').filter(col('csmo202109')!='NaN') \
.filter(col('csmo202110')!='NaN') \
.filter(col('csmo201902')!=0.0).filter(col('csmo201903')!=0.0).filter(col('csmo201904')!=0.0).filter(col('csmo201905')!=0.0) \
.filter(col('csmo201906')!=0.0).filter(col('csmo201907')!=0.0).filter(col('csmo201908')!=0.0).filter(col('csmo201909')!=0.0) \
.filter(col('csmo201910')!=0.0).filter(col('csmo201911')!=0.0).filter(col('csmo201912')!=0.0).filter(col('csmo202001')!=0.0) \
.filter(col('csmo202002')!=0.0).filter(col('csmo202003')!=0.0).filter(col('csmo202004')!=0.0).filter(col('csmo202005')!=0.0) \
.filter(col('csmo202006')!=0.0).filter(col('csmo202007')!=0.0).filter(col('csmo202008')!=0.0).filter(col('csmo202009')!=0.0) \
.filter(col('csmo202010')!=0.0).filter(col('csmo202011')!=0.0).filter(col('csmo202012')!=0.0).filter(col('csmo202101')!=0.0) \
.filter(col('csmo202102')!=0.0).filter(col('csmo202103')!=0.0).filter(col('csmo202104')!=0.0).filter(col('csmo202105')!=0.0) \
.filter(col('csmo202106')!=0.0).filter(col('csmo202107')!=0.0).filter(col('csmo202108')!=0.0).filter(col('csmo202109')!=0.0) \
.filter(col('csmo202110')!=0.0).drop('fraud_year','id_servicio')

In [24]:
df_final.coalesce(1).write.csv("data/fraudes_energuate/result_1_csv", header = True, mode='overwrite')

In [30]:
!rm -fr data/fraudes_energuate/result_1_csv

In [35]:
x = pd.read_csv("data/fraudes_energuate/result_1_csv/part-00000-4c1fe26f-3c95-4c58-81cb-44c02b539b1a-c000.csv")

In [29]:
x.profile_report(minimal=True).to_file("reporte_fraudes.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [9]:
x = pd.read_csv("data/fraudes_energuate/result_1_csv/part-00000-bacccfb8-73fc-4948-8644-daae1e7e41b5-c000.csv")

y = pd.read_csv("data/fraudes_energuate/df_final_no_fraud_csv/part-00000-71ff685d-2696-43ca-8bc7-0ce8b739fbb4-c000.csv")


In [10]:
z = pd.concat([x,y])

In [14]:
y[y['nis_rad']==2004741].head()

,nis_rad,csmo201901,csmo201902,csmo201903,csmo201904,csmo201905,csmo201906,csmo201907,csmo201908,csmo201909,...,csmo202107,csmo202108,csmo202109,csmo202110,ALTURA,DEPARTAMENTO,MUNICIPIO,LOCALIDAD,ID_Centro_Lectura,fraude


In [23]:
len(y)

18999

In [11]:
z[z['fraude']==0]

,nis_rad,csmo201901,csmo201902,csmo201903,csmo201904,csmo201905,csmo201906,csmo201907,csmo201908,csmo201909,...,csmo202107,csmo202108,csmo202109,csmo202110,ALTURA,DEPARTAMENTO,MUNICIPIO,LOCALIDAD,ID_Centro_Lectura,fraude
0,1000064,64,88,74,74,71,92,65,68,67,...,46,35,37,38,1808.0,CHIMALTENANGO,CHIMALTENANGO,CHIMALTENANGO,2201,0
1,1000067,145,103,81,108,80,91,76,105,95,...,103,99,85,98,1807.0,CHIMALTENANGO,CHIMALTENANGO,CHIMALTENANGO,2201,0
2,1000087,86,22,18,44,25,24,14,14,35,...,25,12,14,7,1803.0,CHIMALTENANGO,CHIMALTENANGO,CHIMALTENANGO,2201,0
3,1000123,35,29,3,40,27,27,30,28,28,...,71,68,69,64,1806.0,CHIMALTENANGO,CHIMALTENANGO,CHIMALTENANGO,2201,0
4,1000126,3,6,10,15,6,32,38,15,13,...,117,154,148,146,1801.0,CHIMALTENANGO,CHIMALTENANGO,CHIMALTENANGO,2201,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13147,2180658,83,55,61,68,72,55,62,68,68,...,60,58,64,66,24.0,RETALHULEU,SAN ANDRES VILLA SECA,LINEA C-14,2304,0
13148,2180683,22,32,29,34,31,18,31,33,29,...,111,120,112,100,13.0,RETALHULEU,SAN ANDRES VILLA SECA,LINEA C-14,2304,0
13149,2180691,54,67,68,80,62,4,14,38,38,...,79,81,78,80,24.0,RETALHULEU,SAN ANDRES VILLA SECA,LINEA C-14,2304,0
13150,2180694,99,114,115,107,96,77,97,100,93,...,86,79,90,98,18.0,RETALHULEU,SAN ANDRES VILLA SECA,LINEA C-14,2304,0


In [12]:
df_completa = z.sample(frac=1)

In [22]:
df_completa.to_csv('data/data_final.csv',header = True)

marzo 2021